# OpenCV for real time video object detection
## Applications: road obstacle avoidance, pedestrians, or traffic light detection/classification


In [2]:
import cv2 as cv
# import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage

## Datasets
* https://public.roboflow.com/object-detection/pothole
* https://public.roboflow.com/object-detection/self-driving-car

## Testing Data
https://www.kaggle.com/datasets/robikscube/driving-video-with-object-tracking

## Model Prep/Training

### Prep Training Data

In [3]:
# # Need to manually split Self Driving Car Dataset
# import os
# import shutil
# import random

# # Set the path to your "export" folder
# export_path = "./export"

# # Set the path to the new folders (train, val, test)
# train_path = "./train"
# val_path = "./valid"
# test_path = "./test"

# # Set the split ratios (adjust as needed)
# train_ratio = 0.8  # 80% for training
# val_ratio = 0.1    # 10% for validation
# test_ratio = 0.1   # 10% for testing

# # Create the new folders if they don't exist
# os.makedirs(os.path.join(train_path, "images"), exist_ok=True)
# os.makedirs(os.path.join(train_path, "labels"), exist_ok=True)
# os.makedirs(os.path.join(val_path, "images"), exist_ok=True)
# os.makedirs(os.path.join(val_path, "labels"), exist_ok=True)
# os.makedirs(os.path.join(test_path, "images"), exist_ok=True)
# os.makedirs(os.path.join(test_path, "labels"), exist_ok=True)
# # os.makedirs(val_path, exist_ok=True)
# # os.makedirs(test_path, exist_ok=True)

# # Get the list of image files in the "images" folder
# image_folder_path = os.path.join(export_path, "images")
# image_files = [f for f in os.listdir(image_folder_path) if f.endswith(('.jpg', '.png'))]

# # Randomly shuffle the list of image files
# random.shuffle(image_files)

# # Calculate the number of images for each split
# num_images = len(image_files)
# num_train = int(train_ratio * num_images)
# num_val = int(val_ratio * num_images)
# num_test = int(test_ratio * num_images)

# # Split the image files
# train_images = image_files[:num_train]
# val_images = image_files[num_train:num_train + num_val]
# test_images = image_files[num_train + num_val:]

# # Move the images to their respective folders
# for img in train_images:
#     shutil.move(os.path.join(image_folder_path, img), os.path.join(os.path.join(train_path, "images"), img))

# for img in val_images:
#     shutil.move(os.path.join(image_folder_path, img), os.path.join(os.path.join(val_path, "images"), img))

# for img in test_images:
#     shutil.move(os.path.join(image_folder_path, img), os.path.join(os.path.join(test_path, "images"), img))

# # Repeat the same process for the "labels" folder

# label_folder_path = os.path.join(export_path, "labels")

# # Move the label files to their respective folders
# for lbl in train_images:
#     lbl = lbl.replace('.jpg', '.txt')
#     shutil.move(os.path.join(label_folder_path, lbl), os.path.join(os.path.join(train_path, "labels"), lbl))

# for lbl in val_images:
#     lbl = lbl.replace('.jpg', '.txt')
#     shutil.move(os.path.join(label_folder_path, lbl), os.path.join(os.path.join(val_path, "labels"), lbl))

# for lbl in test_images:
#     lbl = lbl.replace('.jpg', '.txt')
#     shutil.move(os.path.join(label_folder_path, lbl), os.path.join(os.path.join(test_path, "labels"), lbl))


In [9]:
import os

# Define the directory containing your annotation files
annotations_dir = 'ConeTest.v1i.yolov8/train/labels'  # Replace with your actual directory path

# The class ID for pothole that needs to be changed from 0 to 11
old_class_id = 0
new_class_id = 12

# Iterate over all annotation files
for filename in os.listdir(annotations_dir):
    if filename.endswith('.txt'):
        file_path = os.path.join(annotations_dir, filename)
        
        with open(file_path, 'r') as file:
            lines = file.readlines()
        
        updated_lines = []
        
        # Iterate over each line (annotation) and check the class index
        for line in lines:
            parts = line.strip().split()
            class_id = int(parts[0])  # Get the class ID
            
            # If the class ID is the old class (pothole), change it to the new class ID
            if class_id == old_class_id:
                parts[0] = str(new_class_id)  # Update class ID
            
            updated_lines.append(' '.join(parts) + '\n')
        
        # Write the updated lines back to the file
        with open(file_path, 'w') as file:
            file.writelines(updated_lines)

# print("Class ID for potholes has been updated.")


In [15]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

results = model.train(
    data='custom.yaml',
    epochs=30,
)

New https://pypi.org/project/ultralytics/8.3.91 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.81  Python-3.10.16 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=custom.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sa

train: Scanning C:\Users\samue\Documents\projects\ecen331\final\train\labels.cache... 14521 images, 788 backgrounds, 0
val: Scanning C:\Users\samue\Documents\projects\ecen331\final\valid\labels.cache... 2785 images, 96 backgrounds, 0 cor


Plotting labels to runs\detect\train11\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000588, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train11
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/30      2.87G      1.689      1.905      1.159        106        640: 100%|██████████| 908/908 [01:54<00:00, 
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 88/88 [00:1


                   all       2785      24572      0.608      0.299      0.274      0.142

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      30/30       2.2G      1.273     0.7025      1.017         93        640: 100%|██████████| 908/908 [01:39<00:00, 
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 88/88 [00:1

                   all       2785      24572      0.835      0.586      0.667      0.381



30 epochs completed in 0.981 hours.
Optimizer stripped from runs\detect\train11\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train11\weights\best.pt, 6.2MB

Validating runs\detect\train11\weights\best.pt...
Ultralytics 8.3.81  Python-3.10.16 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
Model summary (fused): 72 layers, 3,008,183 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 88/88 [00:1


                   all       2785      24572      0.819      0.582      0.666       0.38
                 biker         44         70      0.732      0.529      0.594       0.33
                   car        963       4783      0.814      0.816      0.858      0.585
            pedestrian        319       1055      0.691      0.595      0.659      0.335
          trafficLight         96        159      0.825      0.836      0.886       0.51
    trafficLight-Green        145        389      0.809      0.674      0.738      0.349
trafficLight-GreenLeft         14         23          1     0.0827      0.226     0.0986
      trafficLight-Red        155        377      0.873      0.743      0.809      0.461
  trafficLight-RedLeft         38         50      0.771       0.58      0.669      0.339
   trafficLight-Yellow          7         12      0.869        0.5       0.79      0.435
trafficLight-YellowLeft          2          2          1          0     0.0622     0.0108
                 tru

In [16]:
metrics = model.val()

Ultralytics 8.3.81  Python-3.10.16 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
Model summary (fused): 72 layers, 3,008,183 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\samue\Documents\projects\ecen331\final\valid\labels.cache... 2785 images, 96 backgrounds, 0 cor
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 175/175 [00


                   all       2785      24572      0.837      0.587      0.667      0.382
                 biker         44         70      0.765      0.529      0.599      0.333
                   car        963       4783      0.818      0.814      0.858      0.585
            pedestrian        319       1055      0.694      0.587      0.659      0.337
          trafficLight         96        159      0.832      0.836      0.883       0.51
    trafficLight-Green        145        389       0.81      0.668      0.738      0.351
trafficLight-GreenLeft         14         23          1     0.0786      0.226     0.0977
      trafficLight-Red        155        377      0.874      0.745      0.812      0.467
  trafficLight-RedLeft         38         50      0.802       0.58      0.669      0.339
   trafficLight-Yellow          7         12          1      0.575      0.799      0.441
trafficLight-YellowLeft          2          2          1          0     0.0603     0.0106
                 tru

In [17]:
eval = model.val(split="test")

Ultralytics 8.3.81  Python-3.10.16 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)


val: Scanning C:\Users\samue\Documents\projects\ecen331\final\test\labels... 1924 images, 104 backgrounds, 0 corrupt: 


val: New cache created: C:\Users\samue\Documents\projects\ecen331\final\test\labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 121/121 [00


                   all       1924      15974      0.829      0.554      0.663      0.387
                 biker         52         90      0.757      0.378      0.464      0.246
                   car        954       4647       0.88      0.781      0.851       0.58
            pedestrian        326       1088       0.79      0.519      0.648      0.314
          trafficLight        112        170      0.852      0.844       0.88      0.551
    trafficLight-Green        172        389      0.846      0.652      0.736      0.343
trafficLight-GreenLeft         18         28          1          0      0.168       0.07
      trafficLight-Red        147        346      0.897      0.755      0.822      0.495
  trafficLight-RedLeft         42         58      0.847      0.397      0.532      0.294
   trafficLight-Yellow          3          5      0.684        0.2      0.479      0.245
                 truck        202        247      0.854      0.755      0.857      0.591
               pothol

## OpenCV

In [20]:
# OpenCV Setup
trainIteration = '11'
modelPath = '/runs/detect/train' + trainIteration + '/weights/best.pt'
print(modelPath)

/runs/detect/train11/weights/best.pt


In [ ]:
cap = cv.VideoCapture('./archive/train/0000f77c-6257be58.mov')

while True:
    ret, frame = cap.read()
    if not ret:
        break

        height, width = frame.shape[:2]

        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

        net.setInput(blob)

        outs = net.forward(output_layers)